In [ ]:
import clips
import sys
sys.path.append('./src/')
from clips_util import print_facts, build_read_assert

# Create the CLIPS environment
env = clips.Environment()

# Define templates
DEFTEMPLATE_PATIENT = """
(deftemplate patient
    (slot name (type STRING))
    (slot has_allergy (type SYMBOL) (allowed-symbols yes no))
    (multislot allergies (type STRING))) ; 'allergies' is a multislot for strings
"""
env.build(DEFTEMPLATE_PATIENT)

DEFTEMPLATE_FOOD = """
(deftemplate food
    (slot name (type STRING))
    (multislot ingredients (type STRING))) ; 'ingredients' is a multislot for strings
"""
env.build(DEFTEMPLATE_FOOD)

DEFTEMPLATE_ALERT = """
(deftemplate alert
    (slot patient (type STRING))
    (slot allergen (type STRING))
    (slot food (type STRING))
    (slot message (type STRING)))
"""
env.build(DEFTEMPLATE_ALERT)

# Corrected Initial facts
DEFFACTS_INITIAL_STATUS = """
(deffacts initial-status
    (patient (name "unknown") (has_allergy no) (allergies)) ; Leave 'allergies' empty
    (food (name "burger") (ingredients "beef" "bun" "lettuce" "tomato"))
    (food (name "salad") (ingredients "lettuce" "tomato" "cucumber"))
    (food (name "pasta") (ingredients "wheat" "tomato" "garlic"))
    (food (name "sandwich") (ingredients "bread" "lettuce" "turkey" "mayo"))
)
"""
env.build(DEFFACTS_INITIAL_STATUS)

# Prompt mapping
prompt_map = {
    "patient:name": "Enter your name: ",
    "patient:has_allergy": "Do you have any allergies? (yes or no): ",
    "patient:allergies": "What kind of allergies do you have? (list space-separated): ",
    "food:name": "Choose a food from the list above. Enter the food name: ",
}

# Build read_assert for interactive inputs
build_read_assert(env, prompt_map)

# Rule to read patient details
DEFRULE_READ_PATIENT_DETAILS = """
(defrule read-patient-details
    =>
    (read_assert patient)
)
"""
env.build(DEFRULE_READ_PATIENT_DETAILS)

# Rule to read food choice
DEFRULE_READ_FOOD_CHOICE = """
(defrule read-food-choice
    =>
    (println "Available foods are: burger, salad, pasta, sandwich")
    (read_assert food)
)
"""
env.build(DEFRULE_READ_FOOD_CHOICE)

# Rule to check food against patient allergies
DEFRULE_CHECK_FOOD = """
(defrule check-food
    (patient (name ?pname) (allergies $?allergies))
    (food (name ?foodname) (ingredients $?ingredients))
    =>
    (bind ?common-allergens (intersection$ ?allergies ?ingredients))
    (if (neq (length$ ?common-allergens) 0)
        then
        (bind ?allergen-str "")
        (foreach ?allergen ?common-allergens
            (bind ?allergen-str (str-cat ?allergen-str ?allergen " ")))
        (assert (alert
            (patient ?pname)
            (allergen ?allergen-str)
            (food ?foodname)
            (message (str-cat "Uh oh, " ?pname "! You can't have this! This food contains: " ?allergen-str))))
        (println "_________________________")
        (println "Alert: Uh oh, " ?pname "! You can't have this! This food contains: " ?allergen-str)
        (println "_________________________"))
    else
        (assert (alert
            (patient ?pname)
            (allergen "None")
            (food ?foodname)
            (message (str-cat "This food does NOT contain any allergen, " ?pname "! Enjoy your food :)")))
        (println "_________________________")
        (println "This food does NOT contain any allergen, " ?pname "! Enjoy your food :)")
        (println "_________________________")))
"""
env.build(DEFRULE_CHECK_FOOD)

# Initialize and run
env.reset()
env.assert_string("(initial-status)")  # Assert initial facts
env.run()

# Print final facts to see system state
print_facts(env)

Enter your name:  Jane Doe
Do you have any allergies? (yes or no):  yes
What kind of allergies do you have? (list space-separated):  peanut


Invalid inputs provided for patient:
invalid cardinality for slot 'allergies'
